geotiff_seaice.py                                                   <br>
                                                                    <br>
 This code gives examples converting variables in AMSR-2 NRT SeaIce <br>
   products in HDF-EOS5 format to GeoTiff, with the use of GIBS     <br>
   color scheme for the variable.                                   <br>
                                                                    <br>
 For other variables (brightness temperature in other wavebands, in <br>
  acsending, descending or day composite mode), similar practice    <br>
  as for the '85H_DAY' band can be applied with setting             <br>
  useGIBScolors = False                                             <br>
                                                                    <br>
 Input: AMSR-2 NRT SeaIce12km file in NSIDC's N/S Polar grids       <br>
 Output: 2 GeoTIFF files for the selected variable in N. and S.     <br>
         polar grids, respectively.                                 <br>
                                                                    <br>
 Required Python packages:                                          <br>
     numpy, pyproj, gdal and GHRC's GIBScolors_utils                <br>
                                                                    <br>
 Note:                                                              <br>
   User can set <useGIBScolors> to True/False to elect use of GIBS  <br>
   color scheme or not.                                             <br>
   In the case of useGIBScolors=True, <gibsMapDir> the location of  <br>
   GIBS color scheme files needs to be provided                     <br>
                                                                    <br>
 By Y. WU of AMSR team @GHRC, May 2021                              <br>

In [ ]:
! pip install numpy
! conda install -c conda-forge gdal
! conda install -c conda-forge pyproj

import os
import numpy as np
from osgeo import gdal, osr
from pyproj import Proj, transform
from GIBScolors_utils import *

#epsg ='3413' &, '3976' for Npolar and Spolar (Polar stereo projection)

npPrj4 ='+proj=stere +lat_0=90 +lat_ts=70 +lon_0=-45 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs '
spPrj4 ='+proj=stere +lat_0=-90 +lat_ts=-70 +lon_0=0 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'

# File name and initial run parameters

h5file = 'C:/Users/lwang/Documents/AMSR/data/AMSR_U2_L3_SeaIce25km_R04_20210622.he5'
fileParts =h5file.split('_')
tstr = fileParts[-1].split('.')[0]

#  Specify variable to be converted
#   Note that there are Tb of different wavebands and polarization in SeaIce12 and SeaIce25, too
# extract ProdStr from filename, i.e., 'SeaIce6', 'SeaIce12', 'SeaIce25'

gcsize = os.path.basename(h5file).split('SeaIce')[1].split('km')[0]
ProdStr = 'SeaIce'+gcsize

if ProdStr == 'SeaIce6':
   vname_partial = '89H_DAY'
else:#SeaIce12 or SeaIce25
   vname_partial = 'ICECON_DAY'
   
# Option to use GIBS color scheme

useGIBScolors = True
gibsMapDir='C:/Users/lwang/Documents/AMSR/colormaps/' #<-- ends with '/' for colormap files parsing

# Open hdf5 with GDAL and get data attributes

hdf_ds = gdal.Open(h5file, gdal.GA_ReadOnly)
dslist = hdf_ds.GetSubDatasets()
ds1 = [s[0].split('//HDFEOS/GRIDS/')[-1] for s in dslist] # include lat, lon

#Pick hemisphere/grid to process
#   NOTE: have to put N. and S. hemisphere variables in separate files
#         as they are in different geogrids

hems=['N','S']
gcsize = gcsize.zfill(2)+'km'
for hem in hems:
    if(hem =='N'):
        polar, hemIDF  = 'Np', 'NH'   # for N. hemisphere
        polarPrj4 = npPrj4
    else:
        polar, hemIDF  = 'Sp', 'SH'   # for S. hemisphere
        polarPrj4 = spPrj4
    SIPrj = osr.SpatialReference()
    SIPrj.ImportFromProj4(polarPrj4)
    vname = '_'.join(['SI',gcsize,hemIDF,vname_partial])
    Pgrid = polar+'PolarGrid'+gcsize

    #---get (lat, lon) for the grid and geotransform info for geotiff
    for ii, a in enumerate(ds1):
        if(Pgrid+'/lat' in a): ilat = ii
        if(Pgrid+'/lon' in a): ilon = ii
    Lat = gdal.Open(hdf_ds.GetSubDatasets()[ilat][0], gdal.GA_ReadOnly)
    Lon = gdal.Open(hdf_ds.GetSubDatasets()[ilon][0], gdal.GA_ReadOnly)
    Lat = Lat.ReadAsArray()
    Lon = Lon.ReadAsArray()
    NY, NX = Lon.shape
    projectn = Proj(polarPrj4)
    X,Y = projectn(Lon,Lat,inverse=False)
    xmin, ymin, xmax, ymax = [X.min(), Y.min(), X.max(), Y.max()]
    xres = (xmax - xmin) / float(NX-1)
    yres = (ymax - ymin) / float(NY-1)

    #---geotransfer info for geotiff
    geotransform = (xmin, xres, 0, ymax, 0, -yres)

    #---get data from the original hdf dataset
    for ii, a in enumerate(ds1):
        if(vname in a): isubset = ii
    varband = gdal.Open(hdf_ds.GetSubDatasets()[isubset][0], gdal.GA_ReadOnly)
    bandArr = varband.ReadAsArray().astype('uint8')

    #-----------------------------------------------
    #- set driver, projection, geotransformation   -
    #-  for output geotiff                         -
    #-----------------------------------------------
    #<--- output geotiff file name with dir path
    TIFFpath= 'C:/Users/lwang/Documents/AMSR/Data_Out/'+vname+'_'+tstr+'.tif'
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(TIFFpath,
                           NX, NY,
                           1,
                           gdal.GDT_Byte)
    out_ds.SetProjection(SIPrj.ExportToWkt())
    out_ds.SetGeoTransform(geotransform)

    #---activate output band
    outBand = out_ds.GetRasterBand(1)
    notVal = 0 # same as fill_value in original file

    #---If opted, use GIBS colors to set colortable
    scaleFC = {}
    if(useGIBScolors): 
        gibsColors = gibsScheme(gibsMapDir, ProdStr, vname, scaleFC)
        if gibsColors:
            colors = colorTable(gibsColors, 255, scaleFC, verb=False)
            outBand.SetColorTable(colors)
            outBand.SetColorInterpretation(gdal.GCI_PaletteIndex)

    #---wrting geotiff band
    outBand.WriteArray(bandArr)
    outBand.SetDescription(vname)
    outBand.SetNoDataValue(notVal)
    outBand = None
    out_ds = None  
    print('Save to',TIFFpath,'\n')
    